#### 01. Import Dependencies


In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

#### 02. Basic Processing

In [3]:
df = pd.read_csv('data/processed/ChurnModelling_Missing_Values_Handled.csv')
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42.00,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41.00,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42.00,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,38.91,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43.00,2,125510.82,1,1,1,79084.10,0


In [4]:
print(f"No of Rows : {len(df)}")

No of Rows : 10000
